In [ ]:
# @title ГОДЖО 30 СЕК — МОДЕЛИ ИЗ ДАТАСЕТА (0 СЕКУНД НА СКАЧИВАНИЕ)
import os
import subprocess
import json
import time
import requests
from IPython.display import FileLink, display

# === ПАПКИ ===
WORKSPACE = "/kaggle/working"
COMFY_DIR = f"{WORKSPACE}/ComfyUI"
OUTPUT_DIR = f"{COMFY_DIR}/output"
INPUT_DIR = f"{WORKSPACE}/input"
FRAMES_DIR = f"{WORKSPACE}/frames"
DATASET_DIR = "/kaggle/input/comfyui-models-gojo"  # ТВОЙ ДАТАСЕТ

# Создаем только базовые папки, ComfyUI папки создадим позже
os.makedirs(INPUT_DIR, exist_ok=True)
os.makedirs(FRAMES_DIR, exist_ok=True)

# === УСТАНОВКА COMFYUI (ОДИН РАЗ) ===
if not os.path.exists(f"{COMFY_DIR}/main.py"):
    print("Установка ComfyUI...")
    # Удаляем папку если она существует но пустая/неполная
    if os.path.exists(COMFY_DIR):
        subprocess.run(["rm", "-rf", COMFY_DIR], check=True)
    subprocess.run(["git", "clone", "https://github.com/comfyanonymous/ComfyUI", COMFY_DIR], check=True)

os.chdir(COMFY_DIR)

# Теперь создаем папки моделей внутри ComfyUI
os.makedirs(f"{COMFY_DIR}/models/checkpoints", exist_ok=True)
os.makedirs(f"{COMFY_DIR}/models/animatediff_models", exist_ok=True)
os.makedirs(f"{COMFY_DIR}/models/upscale_models", exist_ok=True)
os.makedirs(OUTPUT_DIR, exist_ok=True)

# === КОПИРУЕМ МОДЕЛИ ИЗ ДАТАСЕТА (0 СЕКУНД ОЖИДАНИЯ) ===
print("Копируем модели из датасета...")
!cp "$DATASET_DIR/sd_xl_base_1.0.safetensors" "$COMFY_DIR/models/checkpoints/" -f
!cp "$DATASET_DIR/mm_sd_v15_v2.ckpt" "$COMFY_DIR/models/animatediff_models/" -f
!cp "$DATASET_DIR/4x-UltraSharp.pth" "$COMFY_DIR/models/upscale_models/" -f
print("Модели готовы!")

# Устанавливаем зависимости если requirements.txt есть, но пакеты не установлены
if not os.path.exists(f"{COMFY_DIR}/.deps_installed"):
    print("Установка зависимостей...")
    subprocess.run([
        "pip", "install", "-q", "--no-cache-dir",
        "torch", "torchvision", "torchaudio",
        "--extra-index-url", "https://download.pytorch.org/whl/cu121"
    ], check=True)
    subprocess.run(["pip", "install", "-q", "--no-cache-dir", "-r", "requirements.txt"], check=True)
    subprocess.run(["pip", "install", "-q", "huggingface_hub"], check=True)

    # Устанавливаем custom nodes
    os.makedirs("custom_nodes", exist_ok=True)
    os.chdir("custom_nodes")
    if not os.path.exists("ComfyUI-Manager"):
        subprocess.run(["git", "clone", "https://github.com/ltdrdata/ComfyUI-Manager"], check=True)
    if not os.path.exists("ComfyUI-AnimateDiff-Evolved"):
        subprocess.run(["git", "clone", "https://github.com/Kosinkadink/ComfyUI-AnimateDiff-Evolved"], check=True)

    os.chdir(COMFY_DIR)
    # Создаем маркер что зависимости установлены
    with open(f"{COMFY_DIR}/.deps_installed", "w") as f:
        f.write("OK")
    print("Зависимости установлены!")

os.chdir(COMFY_DIR)

# === WORKFLOW: 8 КАДРОВ (упрощенный для теста) ===
# Сначала проверим базовую генерацию без AnimateDiff
workflow = {
  "3": { "class_type": "CheckpointLoaderSimple", "inputs": { "ckpt_name": "sd_xl_base_1.0.safetensors" } },
  "5": { "class_type": "EmptyLatentImage", "inputs": { "width": 512, "height": 768, "batch_size": 8 } },
  "6": { "class_type": "CLIPTextEncode", "inputs": { "text": "cinematic gojo satoru, white spiky hair, blindfold, confident smirk, anime style, detailed, 8k, dynamic lighting", "clip": ["3", 1] } },
  "7": { "class_type": "CLIPTextEncode", "inputs": { "text": "blurry, deformed, low quality, watermark, text, bad anatomy", "clip": ["3", 1] } },
  "8": { "class_type": "KSampler", "inputs": { "seed": 42, "steps": 20, "cfg": 7, "sampler_name": "euler", "scheduler": "normal", "positive": ["6", 0], "negative": ["7", 0], "model": ["3", 0], "latent_image": ["5", 0], "denoise": 1.0 } },
  "9": { "class_type": "VAEDecode", "inputs": { "samples": ["8", 0], "vae": ["3", 2] } },
  "10": { "class_type": "UpscaleModelLoader", "inputs": { "model_name": "4x-UltraSharp.pth" } },
  "11": { "class_type": "ImageUpscaleWithModel", "inputs": { "image": ["9", 0], "upscale_model": ["10", 0] } },
  "12": { "class_type": "SaveImage", "inputs": { "images": ["11", 0], "filename_prefix": "GOJO_FRAMES" } }
}

print("Workflow создан (8 кадров, upscale, без AnimateDiff)")

# === API: ЗАПУСК СЕРВЕРА ===
print("Запуск ComfyUI сервера...")
server = subprocess.Popen(
    ["python", "main.py", "--listen", "127.0.0.1", "--port", "8188"],
    cwd=COMFY_DIR,
    stdout=subprocess.PIPE,
    stderr=subprocess.STDOUT,
    text=True,
    bufsize=1
)

# Читаем вывод сервера в отдельном потоке
import threading
def print_server_output():
    for line in server.stdout:
        print(line, end='')

output_thread = threading.Thread(target=print_server_output, daemon=True)
output_thread.start()

print("Ожидание запуска сервера (может занять 1-2 минуты)...")
server_ready = False
for i in range(180):  # 3 минуты максимум
    try:
        response = requests.get("http://127.0.0.1:8188/system_stats", timeout=2)
        if response.status_code == 200:
            print(f"\n✓ Сервер готов! (прошло {i} сек)")
            server_ready = True
            break
    except requests.exceptions.ConnectionError:
        pass  # Сервер еще не готов
    except Exception as e:
        print(f"Ошибка проверки: {e}")

    if i % 10 == 0 and i > 0:
        print(f"Ожидание... ({i} сек)")
    time.sleep(1)

if not server_ready:
    print("ОШИБКА: Сервер не запустился за 3 минуты.")
    print("Последние строки вывода сервера:")
    server.terminate()
    exit()

# === ОТПРАВКА ЗАДАЧИ ===
print("Генерация 8 кадров (1 сек)...")
try:
    resp = requests.post("http://127.0.0.1:8188/prompt", json={"prompt": workflow, "client_id": "gojo"}, timeout=10)
    resp_data = resp.json()

    if "prompt_id" not in resp_data:
        print("ОШИБКА API:", resp_data)
        server.terminate()
        exit()

    prompt_id = resp_data["prompt_id"]
    print(f"Задача отправлена: {prompt_id}")

except requests.exceptions.RequestException as e:
    print(f"ОШИБКА соединения с сервером: {e}")
    server.terminate()
    exit()

# Ждем завершения генерации
print("Ожидание завершения генерации...")
for i in range(300):  # 10 минут максимум
    try:
        hist = requests.get(f"http://127.0.0.1:8188/history/{prompt_id}", timeout=2).json()
        if prompt_id in hist:
            status = hist[prompt_id].get("status", {})
            if status.get("completed"):
                print("✓ 8 кадров сгенерированы!")
                break
            elif "status_str" in status:
                if i % 10 == 0:
                    print(f"Статус: {status.get('status_str', 'Processing...')}")
    except Exception as e:
        if i % 30 == 0:
            print(f"Ошибка проверки статуса: {e}")
    time.sleep(2)
else:
    print("Таймаут генерации (10 минут)")

server.terminate()
server.wait(timeout=5)

# === ПОДГОТОВКА КАДРОВ ДЛЯ RIFE ===
print("Поиск сгенерированных кадров...")
frame_files = sorted([f for f in os.listdir(OUTPUT_DIR) if f.startswith("GOJO_FRAMES") and f.endswith(".png")])

if len(frame_files) < 8:
    print(f"ОШИБКА: Недостаточно кадров. Найдено {len(frame_files)}, нужно минимум 8.")
    print("Содержимое OUTPUT_DIR:")
    !ls -la $OUTPUT_DIR
    exit()

print(f"Найдено {len(frame_files)} кадров. Копирование в frames/...")
for i, frame_file in enumerate(frame_files[:8]):
    src = f"{OUTPUT_DIR}/{frame_file}"
    dst = f"{FRAMES_DIR}/frame_{i:03d}.png"
    !cp "$src" "$dst"

print("Кадры готовы для RIFE.")

# === RIFE: 8 → 256 КАДРОВ (32 СЕК) ===
print("Интерполяция до 32 секунд...")
!rm -rf /kaggle/working/RIFE
!git clone https://github.com/hzwer/arXiv2020-RIFE /kaggle/working/RIFE
%cd /kaggle/working/RIFE
!pip install -q torch torchvision

!mkdir -p RIFE_trained_model_v4.6
!wget -q -O RIFE_trained_model_v4.6.zip "https://drive.google.com/uc?id=1i3xlKb7ax46jJacQ1aT5vF5lA0Yd5GGI"
!unzip -q RIFE_trained_model_v4.6.zip -d RIFE_trained_model_v4.6

!python inference_video.py \
    --img $FRAMES_DIR \
    --exp=5 \
    --video /kaggle/working/GOJO_30SEC.mp4 \
    --fps=8

# === РЕЗУЛЬТАТ ===
final_video = "/kaggle/working/GOJO_30SEC.mp4"
if os.path.exists(final_video):
    print("ГОТОВО! 30+ СЕКУНД ШОРТС:")
    display(FileLink(final_video))
else:
    print("RIFE не сработал. Логи:")
    !ls $FRAMES_DIR | head -5